In [2]:
import os
import pandas as pd
from glob import glob

In [3]:
def process_nucleosome_counts(bin_dir, intersect_folder):
    """
    Generates a count matrix for nucleosome sites from BEDTools intersect files.

    Parameters:
    - bin_dir: Path to bin directory (e.g., "125_155").
    - intersect_folder: Subfolder inside "bedtools_out" for nucleosome intersect files.

    Returns:
    - Pandas DataFrame containing the nucleosome count matrix.
    """
    intersect_dir = os.path.join(bin_dir, "bedtools_out", intersect_folder)
    bed_files = glob(os.path.join(intersect_dir, "*_nucIntersect.bed"))

    if not bed_files:
        raise FileNotFoundError(f"No nucleosome intersect files found in {intersect_dir}")

    df_list = []

    print(f"Processing nucleosome data in: {intersect_dir}")

    for bedfile in bed_files:
        # Extract sample ID from filename (e.g., "A14891_125_155_nucIntersect.bed" -> "A14891_125_155")
        sample_id = os.path.basename(bedfile).replace("_nucIntersect.bed", "")
        print(f"Reading {sample_id}")

        # Read nucleosome data
        df = pd.read_csv(
            bedfile,
            sep='\t',
            header=None,
            names=["chr", "start", "end", "nuc_id", "peak1", "peak2", "unknown", "count"],
            dtype={"chr": str, "start": int, "end": int, "nuc_id": str, "count": int},  # Enforce correct data types
            na_values=[".", "NA"]  # Handle missing values
        )

        # Keep only relevant columns
        df_final = df[["chr", "start", "end", "nuc_id", "count"]].copy()
        df_final["sample"] = sample_id  # Attach sample info

        df_list.append(df_final)

    print("Finished reading files. Combining data...")

    # Combine all data into one DataFrame
    combined = pd.concat(df_list, ignore_index=True)

    # Pivot to wide format for DESeq2 compatibility
    wide_counts = combined.pivot_table(
        index=["chr", "start", "end", "nuc_id"],
        columns="sample",
        values="count",
        fill_value=0
    ).reset_index()

    print("Finished creating nucleosome count matrix.")
    return wide_counts

In [6]:
# Define input parameters
parent = "/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences"
bin_path = os.path.join(parent, "125_155")
intersect_folder = "nuclIntersect"


# Process nucleosome data
nuc_counts = process_nucleosome_counts(bin_path, intersect_folder)

Processing nucleosome data in: /Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/125_155/bedtools_out/nuclIntersect
Reading A14891_125_155
Reading A14895_125_155
Reading A14899_125_155
Reading A14902_125_155
Reading A14906_125_155
Reading A14903_125_155
Reading A14894_125_155
Reading A14898_125_155
Reading A14901_125_155
Reading A14905_125_155
Reading A14892_125_155
Reading A14896_125_155
Reading A14897_125_155
Reading A14893_125_155
Reading A14904_125_155
Reading A14900_125_155
Finished reading files. Combining data...
Finished creating nucleosome count matrix.


In [9]:
nuc_counts.describe()

sample,start,end,A14891_125_155,A14892_125_155,A14893_125_155,A14894_125_155,A14895_125_155,A14896_125_155,A14897_125_155,A14898_125_155,A14899_125_155,A14900_125_155,A14901_125_155,A14902_125_155,A14903_125_155,A14904_125_155,A14905_125_155,A14906_125_155
count,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07,1.010726e+07
mean,8.141194e+07,8.141204e+07,4.155588e-01,3.746469e-01,6.730973e-01,4.902406e-01,8.524121e-01,5.977637e-01,4.149668e-01,4.827860e-01,4.089351e-01,2.781095e-01,6.894944e-01,4.784596e-01,1.384801e+00,3.829334e-01,4.241418e-01,3.821919e-01
std,5.543588e+07,5.543588e+07,1.044076e+00,9.866529e-01,1.412143e+00,1.156854e+00,1.660728e+00,1.322071e+00,1.031164e+00,1.135133e+00,1.048791e+00,8.207827e-01,1.463545e+00,1.127831e+00,2.382858e+00,9.862623e-01,1.045130e+00,9.738665e-01
min,2.100000e+01,1.210000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.590860e+07,3.590870e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,7.288680e+07,7.288690e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.169878e+08,1.169879e+08,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.492022e+08,2.492023e+08,4.670000e+02,4.570000e+02,7.970000e+02,6.200000e+02,9.630000e+02,6.850000e+02,4.760000e+02,5.750000e+02,5.240000e+02,3.340000e+02,9.190000e+02,5.550000e+02,1.678000e+03,4.660000e+02,4.430000e+02,4.310000e+02


In [ ]:
# Save output
output_path = os.path.join(parent, "ctMatrices", "nuclCounts", "125_155_nucleosome_counts.tsv")
nuc_counts.to_csv(output_path, sep="\t", index=False)

print(f"Saved nucleosome count matrix to {output_path}")

In [10]:
bins = ["125_155","150_180","170_220","220_345","40_150"]
parent = "/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences"
output_dir = "/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/ctMatrices/"

In [11]:
for b in bins:
    bin_path = os.path.join(parent, b)
    mat = process_nucleosome_counts(bin_path, "nuclIntersect")
    outname = os.path.join(output_dir, "nuclCounts", f"{b}_nucleosome_counts.tsv")
    mat.to_csv(outname, sep="\t", index=False)

Processing nucleosome data in: /Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/125_155/bedtools_out/nuclIntersect
Reading A14891_125_155
Reading A14895_125_155
Reading A14899_125_155
Reading A14902_125_155
Reading A14906_125_155
Reading A14903_125_155
Reading A14894_125_155
Reading A14898_125_155
Reading A14901_125_155
Reading A14905_125_155
Reading A14892_125_155
Reading A14896_125_155
Reading A14897_125_155
Reading A14893_125_155
Reading A14904_125_155
Reading A14900_125_155
Finished reading files. Combining data...
Finished creating nucleosome count matrix.
Processing nucleosome data in: /Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/150_180/bedtools_out/nuclIntersect
Reading A14903_150_180
Reading A14898_150_180
Reading A14894_150_180
Reading A14899_150_180
Reading A14891_150_180
Reading A14895_150_180
Reading A14902_150_180
Reading A14906_150_180
Reading A14897_150_180
Reading A14893_150_180
Reading A14904_150_180
Reading A14900_150_180
Readi